In [1]:
from pathlib import Path
from ultralytics import YOLO
import os

In [2]:
# import pretrained model from cache

# ==============================
#           Parameters
# ==============================
pretrained_dir = 'pretrained/' #  directory for pretrained model
model_name = 'yolo11n' # model name
dataset_dir = 'datasets'
dataset_yaml = f'{dataset_dir}/blind-assistance-system/data.yaml' # config file of dataset
imagesize = 640 # image size for trianing and inference
num_epochs = 10 # Training epochs
batch_size = 16 # Batch size
seed = 50

In [3]:
# load pretrianed model
pretrained_model_dir_path = Path(pretrained_dir)
os.makedirs(pretrained_model_dir_path,exist_ok=True)
model_path = pretrained_model_dir_path / f'{model_name}.pt'
model = YOLO(model=model_path)

In [4]:

# ==============================
#             Train
# ==============================

# train with data from Roboflow Data - Blind assistance system 

result = model.train(
    data=Path(dataset_yaml),
    epochs=num_epochs,
    imgsz=imagesize,
    batch=batch_size,
    name='sample',
    seed=seed, # fix seed
    deterministic=True, # reproducible
) 


Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/blind-assistance-system/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=pretrained/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=sample, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

In [5]:
# ==============================
#       Export Best Model
# ==============================

# load best model
try:
    best_model_path = result.save_dir / 'weights' / 'best.pt' 
except:
    best_model_path = Path('runs/detect/sample/weights/best.pt')
    
model = YOLO(best_model_path)

# export to onnx model 
# dynamic: False - fix model input size
model.export(format="onnx", imgsz=imagesize, dynamic=False, simplify=True)



Ultralytics 8.3.221 🚀 Python-3.11.14 torch-2.9.0+cu128 CPU (12th Gen Intel Core i7-12700F)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/home/bee/projects/yolo/runs/detect/sample/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (5.2 MB)

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.72...


2025-10-29 00:37:25.422244608 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


ONNX: export success ✅ 0.6s, saved as '/home/bee/projects/yolo/runs/detect/sample/weights/best.onnx' (10.1 MB)

Export complete (0.8s)
Results saved to /home/bee/projects/yolo/runs/detect/sample/weights
Predict:         yolo predict task=detect model=/home/bee/projects/yolo/runs/detect/sample/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/home/bee/projects/yolo/runs/detect/sample/weights/best.onnx imgsz=640 data=datasets/blind-assistance-system/data.yaml  
Visualize:       https://netron.app


'/home/bee/projects/yolo/runs/detect/sample/weights/best.onnx'